Grupo:



*   Carlos Henrique dos Santos
*   Gabriel Scholze Rosa
*   João Vitor Brandão
*   Matheus Leindorf Muller









In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 63.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9141d2add7b7a356208392a4aa1604b1656a6a624a24dacba9392c1fd457112f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles

spark = SparkSession.builder.master('local[*]').\
        appName('Muller').getOrCreate()
sc = spark.sparkContext

In [ ]:
sc.addFile('/content/drive/MyDrive/transactions_amostra.csv')

In [ ]:
rdd = sc.textFile('file://' + SparkFiles.get('transactions_amostra.csv'))

In [ ]:
# slip de sentenças em palavras
palavras = rdd.map(lambda frase: frase.split(';')).filter(lambda x: x[1].isnumeric()) 

In [ ]:
# questao 1
# We did a filter to select only "Brazil" between the countries, then we utilized a map to add "1" for each "Brazil" found,
# Finnaly, We did a ReduceByKey to sum the values by key. 
palavras.filter(lambda x: x[0]=='Brazil').map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x+y).coalesce(1).saveAsTextFile('Questao1')

In [ ]:
# questao 2
# We did a map to add "1" for each year found, then a ReduceByKey was made to sum all the different years.
palavras.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x+y).coalesce(1).saveAsTextFile('Questao2')


In [ ]:
# questao 3
# We did a map to add "1" for each year and flow type found, then a ReduceByKey was made to sum all the diferrent occorence of years and flow types
palavras.map(lambda x: (x[1] + " " + x[4], 1)).reduceByKey(lambda x, y: x+y).coalesce(1).saveAsTextFile('Questao3')

In [ ]:
# questao 4
# We did a map to join the year, price and a "1" to count later. Then we did a ReduceByKey to sum all the prices, and count them. 
# Finally, we made a MapValues to divide the sum of the prices by their counting (Making the Avarage).
palavras.map(lambda x: (x[1], (float(x[5]),1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1]).coalesce(1).saveAsTextFile('Questao4')

In [ ]:
# questao 5
# We did a filter to select only "Brazil" and "Export" between the countries and flow types. Then we did a map to join the year, unit and category as key; Price as value and a "1" to count later.
# Then we did a ReduceByKey  to sum all the prices, and count them. Finally, we made a MapValues to divide the sum of the prices by their counting (Making the Avarage).
palavras.filter(lambda x: x[0]=="Brazil" and x[4]=="Export").map(lambda x: (x[1]+" "+ x[7]+ " "+x[9],(float(x[5]),1)))\
.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1]).coalesce(1).saveAsTextFile('Questao5')

In [ ]:
# questao 6
# We did a map to join the unit and the year as a key, and 3 prices for different operations: the maximum, the minimum and the mean as values, and a "1" to count later. 
# Then we did a ReduceByKey to get the maximum, the minimum and the mean prices. Finally we made a map to put the items in order, and get the mean by dividing the sum of prices by the count of them.
palavras.map(lambda x: (x[7] + " "+x[1],(float(x[5]),float(x[5]),float(x[5]),1.0)))\
.reduceByKey(lambda x,y: (x[0] if x[0]>y[0] else y[0],\
                          x[1] if x[1]<y[1] else y[1],\
                           x[2]+y[2], x[3]+y[3]))\
                           .map(lambda x: (x[0], (x[1][0],x[1][1], x[1][2]/x[1][3]))).coalesce(1).saveAsTextFile('Questao6')

In [ ]:
# questao 7
# Fisrt we did a filter to select only "2016" between the years. Then we did a map to join the Flow with the Commodity as key, and the Amount as Value. 
# Then we did a GroupByKey to organize and another map to get the Flow type as key, and the sum of Amount with Commodity as Values.
# Finally we did a ReduceByKey to get only the most commercialized commodity in 2016 by Flow Type
palavras.filter(lambda x: x[1]=="2016")\
.map(lambda x: ((x[4],x[3]),float(x[8])))\
.groupByKey().map(lambda x: (x[0][0], (sum(x[1]),x[0][1])))\
.reduceByKey(lambda x,y: x if x>y else y).coalesce(1).saveAsTextFile('Questao7')

Grupo:



*   Carlos Henrique dos Santos
*   Gabriel Scholze Rosa
*   João Vitor Brandão
*   Matheus Leindorf Muller